In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score, confusion_matrix

In [2]:
# Loading data
df = pd.read_csv('Dataset/cleaned_adult.csv')

In [3]:
X = df.drop(['income'], axis=1) # Features
y = df['income'] # Label

In [4]:
# Apply Standard Scaler for the features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)